## Implementation of Hybrid ESN for Lorenz 96

### Libraries

In [ ]:
# === Libraries ===
import numpy as np
import pandas as pd
import torch
from torchesn.nn import ESN
from torchesn.nn import Reservoir
from torchesn.utils import prepare_target
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.stats import linregress
import scipy.sparse as sparse
from pathlib import Path

### Data 

In [ ]:
# data = np.load('./data/truth_h_0.5_c_8_F_20.npy')

In [ ]:
X_data = np.load('./data/truth_h_0.5_c_8_F_20.npy')[:, :8]

In [ ]:
data = X_data.T

In [ ]:
# Slice 1000 steps (≈ 22 Lyapunov times)
steps = 1000
X_plot = X_data[:steps].T  # shape: (8, steps)

# Create plot axes
time_axis = np.linspace(0, 9, steps)
space_axis = np.arange(8)

# Plot contour
fig, ax = plt.subplots(figsize=(14, 3))
contour = ax.contourf(time_axis, space_axis, X_plot, levels=50, cmap='bwr', vmin=-2.5, vmax=2.5)

# Labels
ax.set_title("Spatiotemporal Contour of $X_{true}$ (~9 Lyapunov Times)", fontsize=14)
ax.set_xlabel("Timesteps", fontsize=12)
ax.set_ylabel("X", fontsize=12)
ax.set_yticks(np.arange(8))
ax.set_yticklabels([f"$X_{i}$" for i in range(8)])

# Colorbar with custom ticks
cbar = fig.colorbar(contour, ax=ax)
cbar.set_label("$X_{true}$", fontsize=12)
cbar.set_ticks([-2, 0, 2])

plt.tight_layout()
plt.show()


### ESN

In [ ]:
shift_k = 0

approx_res_size = 5000


model_params = {'tau': 0.25,
                'nstep': 1000,
                'N': 8,
                'd': 22}

res_params = {'radius':0.1,
             'degree': 3,
             'sigma': 0.5,
             'train_length': 500000,
             'N': int(np.floor(approx_res_size/model_params['N']) * model_params['N']),
             'num_inputs': model_params['N'],
             'predict_length': 10000,
             'beta': 0.0001
              }

# The ESN functions for training
def generate_reservoir(size,radius,degree):
    sparsity = degree/float(size)
    A = sparse.rand(size,size,density=sparsity).todense()
    vals = np.linalg.eigvals(A)
    e = np.max(np.abs(vals))
    A = (A/e) * radius
    return A

def reservoir_layer(A, Win, input, res_params):
    states = np.zeros((res_params['N'],res_params['train_length']))
    for i in tqdm(range(res_params['train_length'] - 1), desc="Reservoir states"):
        states[:,i+1] = np.tanh(np.dot(A,states[:,i]) + np.dot(Win,input[:,i]))
    return states


def train_reservoir(res_params, data, save_path=None):
    A = generate_reservoir(res_params['N'], res_params['radius'], res_params['degree'])
    q = int(res_params['N'] / res_params['num_inputs'])
    Win = np.zeros((res_params['N'], res_params['num_inputs']))

    for i in range(res_params['num_inputs']):
        np.random.seed(seed=i)
        Win[i*q:(i+1)*q, i] = res_params['sigma'] * (-1 + 2 * np.random.rand(1, q)[0])

    states = reservoir_layer(A, Win, data, res_params)
    Wout = train(res_params, states, data)
    x = states[:, -1]

    if save_path:
        save_esn_weights(A, Win, Wout, save_path)

    return x, Wout, A, Win

def train(res_params,states,data):
    beta = res_params['beta']
    idenmat = beta * sparse.identity(res_params['N'])
    states2 = states.copy()
    for j in range(2,np.shape(states2)[0]-2):
        if (np.mod(j,2)==0):
            states2[j,:] = (states[j-1,:]*states[j-2,:]).copy()
    U = np.dot(states2,states2.transpose()) + idenmat
    Uinv = np.linalg.inv(U)
    Wout = np.dot(Uinv,np.dot(states2,data.transpose()))
    return Wout.transpose()

def predict(A, Win, res_params, x, Wout):
    output = np.zeros((res_params['num_inputs'],res_params['predict_length']))
    for i in range(res_params['predict_length']):
        x_aug = x.copy()
        for j in range(2,np.shape(x_aug)[0]-2):
            if (np.mod(j,2)==0):
                x_aug[j] = (x[j-1]*x[j-2]).copy()
        out = np.squeeze(np.asarray(np.dot(Wout,x_aug)))
        output[:,i] = out
        x1 = np.tanh(np.dot(A,x) + np.dot(Win,out))
        x = np.squeeze(np.asarray(x1))
    return output, x

def save_esn_weights(A, Win, Wout, path="trained_esn.pt"):
    weights = {
        'A': torch.tensor(A, dtype=torch.float32),
        'Win': torch.tensor(Win, dtype=torch.float32),
        'Wout': torch.tensor(Wout, dtype=torch.float32)
    }
    torch.save(weights, path)
    print(f"ESN weights saved to {path}")

def load_esn_weights(path="trained_esn.pt"):
    weights = torch.load(path)
    A = weights['A'].numpy()
    Win = weights['Win'].numpy()
    Wout = weights['Wout'].numpy()
    print(f"ESN weights loaded from {path}")
    return A, Win, Wout

In [ ]:
x, Wout, A, Win = train_reservoir(res_params, data, save_path="trained_esn.pt")

In [ ]:
%whos


In [ ]:
# Run prediction starting from final reservoir state
output, _ = predict(A, Win, res_params, x, Wout)

# Optional: save predictions
np.save("esn_prediction.npy", output)
print("Prediction shape:", output.shape)  # Should be (8, predict_length)

In [ ]:
import pandas as pd

# Load from local directory
df = pd.read_csv('./data/3tier_lorenz_v3.csv', header=None)
print("Shape of the data:", df.shape)


### Hybrid ESN 

### Ensemble Kalman Filtering

### Evaluation Metrics